In [38]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import ollama 
import http.client 
import requests

In [39]:

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY') 
booking_api_key = os.getenv("RAPID_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI() 
DB = "ticket_price"


OpenAI API Key exists and begins sk-proj-


In [40]:

system_message ="You are a helpful assistant for hotel/hostel booking  called BookingAi. " 
system_message +="Give me short answers, no more than 3 sentence. Respond to greetings and general conversations."
system_message +="Always be accurate. If you dont know the answers , simply say no." 
system_message +="When a user asks for information that requires external data or action, use the available tools to get the information specifically."

In [41]:
url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchDestination"   
headers = {
	"x-rapidapi-key": booking_api_key,
	"x-rapidapi-host": "booking-com15.p.rapidapi.com"
}


In [ ]:
def get_hotel_price(city, checkin_date, checkout_date):
    print(f"database tool called for the hotel price of {city}" , flush=True) 
    query_params = {"query":city}
    destination_response= requests.get(url , headers =headers , params=query_params).json() 
    if not destination_response.get("data"):
        return {"error": "No Destination Found"}  

    dest_id = destination_response["data"][0]["dest_id"] 
    dest_type = destination_response["data"][0]["dest_type"]      
    hotels_url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels" 
    hotel_req = requests.get(
        hotels_url,
        headers = headers,
        params={
            "dest_id":dest_id,
            "search_type" : dest_type, 
            "arrival_date":checkin_date,
            "departure_date":checkout_date,
            "adults":1,
            "room_qty":1 ,
            "order_by":"price"
        }
    ).json()
    hotels = []  
    for h in hotel_req.get("data",{}).get("hotels",[])[:5]:
        hotels.append({
             "name": h["property"]["name"],
            "price": h["property"]["priceBreakdown"]["grossPrice"]["value"],
            "currency": h["property"]["priceBreakdown"]["grossPrice"]["currency"],
            "rating": h["property"].get("reviewScore")
        })
    print(hotels)   
    return{
        "city":city,
        "departure_date":checkin_date,
        "departure_date":checkout_date,
        "hotels":hotels
    }    
    

In [43]:
hotel_price_function={
    "name":"get_hotel_price", 
    "description":"Find the hotel price for a city and date range",
    "parameters":{
        "type":"object",
        "properties":{
            "city":{
                "type":"string",
                "description":"City name"
            },
            "checkin_date":{
                "type":"string",
                "description":"Check-in date in YYYY-MM-DD format"
            },
            "checkout_date":{
                "type":"string",
                "description":"Check-out date in YYYY-MM-DD format"
            }
        },
        "required":["city","checkin_date","checkout_date"]
    }
}

In [44]:
tool = [{"type":"function","function":hotel_price_function}]

In [45]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tool)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_call(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tool)
    
    return response.choices[0].message.content


In [ ]:
def handle_tool_call(message): 
    responses = [] 
    for tool_call in message.tool_calls:
        if tool_call.function.name  == "get_hotel_price": 
            arguments = json.loads(tool_call.function.arguments) 
            city = arguments.get("city")
            checkin_date = arguments.get("checkin_date")
            checkout_date = arguments.get("checkout_date") 
            print("city",city)
            hotel_details = get_hotel_price(city,checkin_date,checkout_date) 
            responses.append({
                "role":"tool",
                "content":json.dumps(hotel_details),
                "tool_call_id":tool_call.id
            })
    return responses  


gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


hello ChatCompletionMessageFunctionToolCall(id='call_XCtLw4ozXcMJtl2tlzVMMfaa', function=Function(arguments='{"city":"Berlin","checkin_date":"2025-12-25","checkout_date":"2025-12-29"}', name='get_hotel_price'), type='function')
city Berlin
database tool called for the hotel price of Berlin
hhhh -1746443 city
hello  {'status': True, 'message': 'Success', 'timestamp': 1766495957648, 'data': {'hotels': [{'hotel_id': 9927832, 'accessibilityLabel': 'SpreeBlau Apartments.\n4 out of 5 for property rating.\nThis is a Preferred property.\n8.6 Fabulous 2229 reviews.\n\u200eTreptow-Köpenick\u202c • \u200e12.4 km from centre\u202c.\nLimited-time Deal.\n Entire studio – 30\xa0m² : 1\xa0bed • 1\xa0bedroom • 1\xa0bathroom.\nOriginal price 550 USD. Current price 358 USD..\nIncludes taxes and charges.', 'property': {'name': 'SpreeBlau Apartments', 'blockIds': ['992783202_372235034_1_0_0'], 'currency': 'EUR', 'checkout': {'untilTime': '10:30', 'fromTime': '00:00'}, 'isFirstPage': True, 'id': 9927832, 'c